# PORTUGUESE TWEETS ANALYSIS - training

In [1]:
import spacy
import pandas as pd
from spacy.matcher import Matcher
#spacy.cli.download("pt_core_news_lg")
from spacy import displacy
from sklearn.model_selection import train_test_split
from spacy.tokens import DocBin


In [2]:
with open("C:\\Test folder\\NoThemeTweets.csv") as csv_file:
    data = pd.read_csv(csv_file)

In [3]:
data.head()

,id,tweet_text,tweet_date,sentiment,query_used
0,1031761728445530112,@Tixaa23 14 para eu ir :),Tue Aug 21 04:35:39 +0000 2018,Positivo,:)
1,1031761040462278656,@drexalvarez O meu like eu já dei na época :),Tue Aug 21 04:32:55 +0000 2018,Positivo,:)
2,1031760962372689920,Eu só queria conseguir comer alguma coisa pra ...,Tue Aug 21 04:32:37 +0000 2018,Positivo,:)
3,1031760948250456066,:D que lindo dia !,Tue Aug 21 04:32:33 +0000 2018,Positivo,:)
4,1031760895985246208,"@Primo_Resmungao Pq da pr jeito!!é uma ""oferta...",Tue Aug 21 04:32:21 +0000 2018,Positivo,:)


In [4]:
data = data[['tweet_text','sentiment']].dropna()

In [5]:
data.groupby('sentiment').size()

sentiment
Negativo    522707
Positivo    263107
dtype: int64

## DATA PREPARATION for SPACY

In [9]:
# Creating a train dataset and a validation dataset
train_df = data[:90000].sample(n = 5000)
print("train dataset:", train_df.groupby('sentiment').size())

valid_df = data[90001:200000].sample(n = 5000)
print("valid dataset:",valid_df.groupby('sentiment').size())

train dataset: sentiment
Negativo    1096
Positivo    3904
dtype: int64
valid dataset: sentiment
Negativo    4447
Positivo     553
dtype: int64


In [10]:
# Converting the dataframe into a list of tuples
train_df['tuples'] = train_df.apply(lambda row: (row['tweet_text'],row['sentiment']), axis=1)
train_data = train_df['tuples'].tolist()
#train[:10]
#print(train_data[0])
print(type(train_data))

valid_df['tuples'] = valid_df.apply(lambda row: (row['tweet_text'],row['sentiment']), axis=1)
valid_data = valid_df['tuples'].tolist()
#train[:10]
#print(valid_data[0])
print(type(valid_data))
#print(valid_data)

<class 'list'>
<class 'list'>


In [11]:
nlp = spacy.load("pt_core_news_lg")

In [12]:
def make_docs(data):
    docs = []
    for doc, label in nlp.pipe(data, as_tuples=True):
        if label == 'Negativo':
            doc.cats["positive"] = 0 #Maps a label to a score for categories applied to the document. 
            doc.cats["negative"] = 1
        else:
            doc.cats["positive"] = 1
            doc.cats["negative"] = 0
        docs.append(doc)
    return (docs)

In [13]:
## create training and validation data
num_texts = 800 # amount of validation and training generated

train_docs = make_docs(train_data[:num_texts])
doc_bin = DocBin(docs=train_docs)
doc_bin.to_disk("./C:/Test folder/KnowingSpacy/self learning/trainpt.spacy")

valid_docs = make_docs(valid_data[:num_texts])
doc_bin= DocBin(docs=valid_docs)
doc_bin.to_disk("./C:/Test folder/KnowingSpacy/self learning/validpt.spacy")

In [ ]:
### To train the model

# 1. create a base_config_model and copy data from  https://spacy.io/usage/training#quickstart
# 2. update base_config.cfg file with the train and dev paths

# anaconda prompt
# 3.python -m spacy init fill-config ./base_config.cfg ./config.cfg
# 4. open config
# 3. python -m spacy train config.cfg --output ./output

- Loss:	The training loss representing the amount of work left for the optimizer. Should decrease, but usually not to 0.

- Precision (P):	Percentage of predicted annotations that were correct. Should increase.

- Recall (R):	Percentage of reference annotations recovered. Should increase.

- F-Score (F):	Harmonic mean of precision and recall. Should increase.

- UAS / LAS:	Unlabeled and labeled attachment score for the dependency parser, i.e. the percentage of correct arcs. Should increase.

- Speed:	Prediction speed in words per second (WPS). Should stay stable.